let's test models on new data (from twitter) to check generality of using

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

from spam_classifier.spam_classifier import SpamClassifier
from toxic_classifier.toxic_classifier import get_toxic_classifier
from sentiment_analysis.sentiment_analysis_model import load_model

In [ ]:
def calculate_metrics(y_test, y_pred):
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  print(f"Accuracy = {acc}; f1-score = {f1}")

# download data

In [ ]:
PATH_TO_FOLDER = "drive/MyDrive/dl_project/twitter"

PATH_TO_TEST_SPAM = PATH_TO_FOLDER + "/test_spam.csv"
PATH_TO_TEST_TOXIC = PATH_TO_FOLDER + "/test_toxic.csv"
PATH_TO_TEST_EMOTION = PATH_TO_FOLDER + "/test_emotion.csv"

PATH_TO_TEST = PATH_TO_FOLDER+"/twitter_test.csv"

In [ ]:
test_spam = pd.read_csv(PATH_TO_TEST_SPAM)
test_toxic = pd.read_csv(PATH_TO_TEST_TOXIC)
test_emotion = pd.read_csv(PATH_TO_TEST_EMOTION)
test_mixed = pd.read_csv(PATH_TO_TEST)

# spam

In [ ]:
spam_model = SpamClassifier()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  supe

In [ ]:
test_spam_texts = test_spam.drop(columns = ['is_spam'])
test_spam_probas = spam_model.predict(test_spam_texts)
test_spam_predictions = list(np.where(np.array(test_spam_probas) > 0.5, 1, 0 ))
test_spam_y = test_spam['is_spam']

In [ ]:
calculate_metrics(test_spam_y, test_spam_predictions)

Accuracy = 0.345; f1-score = 0.5130111524163569


In [ ]:
test_spam['predictions'] = test_spam_predictions

show texts with wrong prediction

In [ ]:
wrong_predictions = test_spam[test_spam['predictions'] != test_spam['is_spam']].text

In [ ]:
for i, text in enumerate(wrong_predictions):
  print(f"{i}. {text}")
  if i == 5:
    break

0. DOES ANYONE ANYWHERE ACTUALLY BELIEVE THIS? -- Video in apparent Epstein suicide attempt is lost, US says <em>URL01 Removed</em>
1. Dead animals must be treated with respect.
2. .@USER01 good to meet you today @USER02 The work you do in promoting #mentalhealth and for… <em>URL01 Removed</em>
3. 5am knock with a big red key & Glock... hopefully! Great work #GoodoldGolfHotel<em>URL01 Removed</em> …
4. @USER01 @USER02 Do you know that babies in the womb twist and turn to avoid their murders?
5. So I accidentally (I swear it was an accident!) followed back a cam girl. We I think she’s a cam girl. Now my feed and dms are full of weird shit. I guess I should be happy no dick picks. Just chonch shots. Lol


In [ ]:
test_mixed_texts = pd.DataFrame(test_mixed.text)
test_mixed_probas = spam_model.predict(test_mixed_texts)

In [ ]:
test_mixed_predictions = list(np.where(np.array(test_mixed_probas) > 0.5, 1, 0 ))
test_mixed_y = test_mixed['is_spam']

In [ ]:
calculate_metrics(test_mixed_y, test_mixed_predictions)

Accuracy = 0.7268571428571429; f1-score = 0.4191980558930741


Выводы о spam модели:

- на наших тестовых данных модель показала плохие результаты
- причиной плохих результатов, скорее всего, является специфичность спама для каждой платформы и даже затрагиваемой тематики
- получаем, что полученная модель может быть применена только к некоторому специфичному срезу спама, похожему на тренировочный датасет

# toxic

In [ ]:
toxic_model = get_toxic_classifier()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
class TextsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row["text"]
        return text

In [ ]:
toxic_dataset = TextsDataset(test_toxic)
toxic_dataloader = DataLoader(toxic_dataset, batch_size=16, shuffle=False)
trainer = Trainer()
predictions = trainer.predict(toxic_model, toxic_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [ ]:
help_predictions = []
for batch_res in predictions:
  help_predictions += batch_res

In [ ]:
toxic_prediction = [1 if 1 in x else 0 for x in help_predictions]
toxic_y = test_toxic.is_toxic.to_list()


In [ ]:
calculate_metrics(toxic_y, toxic_prediction)

Accuracy = 0.833; f1-score = 0.8142380422691881


Выводы о модели toxic классификации:

- модель показала достаточно хорошие результаты даже на данных из абсолютно нового датасета
- токсичные комментарии/тексты имеют общие признаки, которые не сильно меняются в зависимости от контекста
- тренировочный датасет скорее всего был достаточно репрезентативным, поэтому позволил выделить многие паттерны "токсичности"

# emotion

In [ ]:
emotion_model = load_model("sentiment_analysis.ckpt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
test_emotion = test_emotion[test_emotion['is_neutral'] != 1].drop(columns = ['is_neutral'])
test_emotion

,text,is_negative,is_positive
2,china worried new stance modi govt\n,0,1
3,just watch fun\nmodi will win election\nhuh,0,1
5,dont worry minister indian muslims are develop...,0,1
7,breaking news jem chief masood azhar dead some...,1,0
8,governor kalyan singh aligarh 23rd march all a...,0,1
...,...,...,...
1495,1960s lal bahadur shatri resigned because trai...,1,0
1496,this what has led global rise and why disconte...,0,1
1497,asshole bahujan radical marxist grow brain kno...,1,0
1498,raghuram rajan sent list high profile bank fra...,1,0


In [ ]:
emotion_dataset = TextsDataset(test_emotion)
emotion_dataloader = DataLoader(emotion_dataset, batch_size=16, shuffle=False)
trainer = Trainer()
predictions = trainer.predict(emotion_model, emotion_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-104-3afc1b0a5ca1>:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.softmax(logits)


In [ ]:
help_predictions = []
for batch_res in predictions:
  help_predictions += batch_res

In [ ]:
emotion_predictions_positive = [1 if i.item() in {1, 2, 5} else 0 for i in help_predictions]
emotion_y = test_emotion.is_positive.to_list()

In [ ]:
calculate_metrics(emotion_y, emotion_predictions_positive)

Accuracy = 0.395; f1-score = 0.5392231530845392


Выводы о sentimental model:
- наша проверка качества, по сути, давала послабление на точность модели (не ожидалось, что она уловит конкретную эмоцию, а только общее настроение сообщения)
- модель на новых данных показала плохое качество
- выражать эмоции в тексте можно разными способами. Из-за этого, вероятно, модель не смогла считывать positive/negative эмоции корректно 
- построить модель с хорошим качеством для анализа эмоций, вероятно, сложно. Причина -- самые различные способы проявления эмоций, которые со временем могут меняться